<a href="https://colab.research.google.com/github/WellersonPrenholato/Pythonisa/blob/main/Pythonisa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pandas
!pip3 install --upgrade gspread
!pip3 install pulp


Requirement already up-to-date: gspread in /usr/local/lib/python3.6/dist-packages (3.6.0)


In [ ]:
from datetime import datetime
import gspread
import pandas as pd
import numpy as np
from pulp import *

class BakeryProblem:
  def __init__(self, From, SheetUrl, Credentials ):
    self.dfRecipes = None
    self.dfResources = None
    self.lpProblem = None
    self.lpVariables = None


    if From == 'gsheet':
        self.dfRecipes = self.dataframeFromSheet(SheetUrl,
            Credentials,
            'measures'
        )

        self.dfResources = self.dataframeFromSheet(SheetUrl,
            Credentials,
            'total_available'
        ) 
        


  
  def dataframeFromSheet(self, Source,
                       credentialsFilename, 
                       worksheet):
    gc = gspread.service_account(credentialsFilename)
    wb = gc.open_by_url(Source)
    data = wb.worksheet(worksheet).get_all_values()
    headers = data.pop(0)

    df = pd.DataFrame(data, columns=headers, dtype=np.float)
  
    return df


  def mountLP(self, title=f"Problem_{datetime.now()}"):
    self.lpProblem = LpProblem(title, LpMaximize)

    # Create variables 
    self.lpVariables = {}
    for produto in self.dfRecipes['PRODUTO']:
      self.lpVariables[produto] = LpVariable(name=produto, 
                                      lowBound=self.dfRecipes.query(f"PRODUTO == '{produto}'").iloc[0]['DEMANDA_LOTE'],
                                      upBound=None,
                                      cat='Integer')
      
    # Create Objective Function
    self.lpProblem += lpSum([ self.lpVariables[produto] * self.dfRecipes.query(f"PRODUTO == '{produto}'").iloc[0]['PRECO_UN'] * self.dfRecipes.query(f"PRODUTO == '{produto}'").iloc[0]['ITENS_LOTE']  
                      for produto in self.dfRecipes['PRODUTO'] ])

    # Create Constraints
    ingredients = [ column for column in self.dfResources.columns 
                  if column not in ['PRODUTO', 'PRECO_UN', 'DEMANDA_DIA', 'DEMANDA_LOTE', 'VALOR_BASE', 'ITENS_LOTE', 'PESO_MEDIO_ITEM', 'TEMPO_MEDIO_PRODUCAO'] and len(column) > 0 ]
    
    for ingredient in ingredients:
        self.lpProblem += lpSum([ self.lpVariables[produto] * self.dfRecipes.query(f"PRODUTO == '{produto}'").iloc[0][ingredient]  for produto in self.dfRecipes['PRODUTO'] ]) <= self.dfResources[ingredient]
    
  def optimize(self):
    try:
      status = self.lpProblem.solve()
      if (  status == 1 ):
        print(f"BAKERY - LP SOLUTION {LpStatus[status]}")
        for (fantasyName,var) in self.lpVariables.items():
            print(f"{fantasyName} = {value(var)}")
        print(f"Profit total: {value(self.lpProblem.objective)}")
      else:
        print("Solution not found.")
    except AttributeError:
      raise AttributeError('LP problem is not defined.')


  def showLP(self):
    print(self.lpProblem)

  def getMinValuesResource(self):
    ingredients = [ column for column in self.dfRecipes.columns if column not in ['PRODUTO', 'PRECO_UN', 'DEMANDA_DIA', 'DEMANDA_LOTE', 'VALOR_BASE', 'ITENS_LOTE', 'PESO_MEDIO_ITEM', 'TEMPO_MEDIO_PRODUCAO'] ]

    for ingredient in ingredients:
      sum = 0
      for i in range(self.dfRecipes.shape[0]):
        sum += self.dfRecipes.at[i,'DEMANDA_LOTE'] * self.dfRecipes.at[i, ingredient]
      print(ingredient,sum )


In [ ]:
myProb = BakeryProblem(From='gsheet', 
                       SheetUrl='https://docs.google.com/spreadsheets/d/1hwY2ab3CWWZzGYIu1Xnj-4vbM3iQM240EsfiFAG50EM',
                       Credentials='sample_data/credentials.json')
myProb.mountLP()
myProb.showLP()
myProb.optimize()
# myProb.getMinValuesResource()
# myProb.showLP()


Problem_2020-12-08_00:30:58.099100:
MAXIMIZE
0.9*pao_trança_de_coco + 3.74*pão_doce_recheado_com_goiabada + 6.0*pão_doce_sem_recheio + 1.5*pão_francês + 0.0
SUBJECT TO
_C1: 800 pao_trança_de_coco + 600 pão_doce_recheado_com_goiabada
 + 500 pão_doce_sem_recheio + 1000 pão_francês <= 180000

_C2: 240 pao_trança_de_coco + 92.4 pão_doce_recheado_com_goiabada
 + 110 pão_doce_sem_recheio <= 5800

_C3: 402.8 pao_trança_de_coco + 300 pão_doce_recheado_com_goiabada
 + 180 pão_doce_sem_recheio <= 25000

_C4: 296 pao_trança_de_coco + 16.2 pão_doce_recheado_com_goiabada
 + 219.34 pão_doce_sem_recheio + 10 pão_francês <= 10000

_C5: 8 pao_trança_de_coco + 3 pão_doce_recheado_com_goiabada
 + 219.75 pão_doce_sem_recheio + 20 pão_francês <= 5000

_C6: 378 pão_doce_recheado_com_goiabada <= 20000

_C7: 112 pao_trança_de_coco + 150 pão_doce_recheado_com_goiabada <= 8000

_C8: 261.18 pão_doce_recheado_com_goiabada + 22.5 pão_doce_sem_recheio
 + 10 pão_francês <= 15000

_C9: 351.43 pao_trança_de_coco <= 50

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
